# Summary Assignment for Module 6
<blockquote>This is an assignment in a Jupyter notebook which will be autograded. To avoid autograder errors, please do not add or delete any cells. Also, <b>run all cells even if they are hidden</b> and not requiring any input from you. You may add additional calulations or print statements to any cell to help you see the current values of variables you may be working with.
</blockquote>

**Rounding Error:** Ideally, your answers will be given from a direct Python calculation. For example, given a $2 \times 2$ array $A$, if you are asked to multiply the $(1,3)$ entry with the $(2,2)$ entry and to store the result in a variable `x`, you would type

<code>x = A[1,3]*A[2,2]</code>

(Recall that Python uses zero-based indexing so the $(1,3)$ entry is actually coming from the second row.)

However, if you compute `A[1,3]*A[2,2]` and see `0.23719445178`, you could choose to type out the answer as, for example,

<code>x = 0.23719445</code>

<u>as long as you keep at least 3 decimal places of accuracy</u>.

In [1]:
# Run this cell to import the NumPy library with the name "np" and a
# testing library that will be used by the autograder
import numpy as np
import numpy.testing as npt

# Problem: Marvin in a Maze

Suppose that Marvin the rat is in a maze, with 14 cells, depicted below. Marvin continuously wanders the maze in search of food. There is an endless supply of cheese in the maze in cell 3 and an endless supply of lettuce in cell 14. There are two blocked cells, indicated in blue-green with diagonal stripes. 

<br>
<img src="rat2_mdp.png" width="400"> <br>

In cells other than 3, 10, and 14, Marvin can choose to go up, down, left, or right. Attempting to go off the maze and into a blocked cell will leave Marvin's position unchanged for the next time step. Upon arriving in cell 3, Marvin will eat some cheese but, in the next move, he will teleport to cell 8 with probability 1.  Upon arriving in cell 14, Marvin will eat some lettuce but, in the next move, he will teleport to cell 12 with probability 1. These teleportations will always happen and there are no policy moves to be learned for exiting cells 3 and 14. Although Marvin is happy to have found some food, he always wants more and will continue wander indefinitely!

When moving out of cell 10, with probability 1/2, Marvin will teleport to cell 5. The rest of the time, he will choose from neighboring cells according to the policy he is learning on his journey.

Rewards are as follows.

<ul>
    <li> If Marvin enters cell 3, he receives a reward of +5 because cheese is great!</li>
    <li> If he enters cell 14, he receives a reward of +2 because lettuce is better than nothing.</li>
    <li> If he attempts to go off the grid or into a blocked cell, he receives a reward of -1.</li>
    <li> When he enters any other cells, either by walking or teleportation, he receives a reward of 0.</li>
</ul>

Using a discount factor of $0.95$, find the optimal policy for Marvin's movements. Begin with a "mostly random walk policy" where up, down, left, and right movements, are equally likely unless Marvin is in a special cell.

Special Transition Probability Examples:
<ul>
   <li>If he is in cell 1, he will move to the right to cell 2 with probability 1/4, he will move down to cell 5 with probability 1/4, and he will stay in cell 1 with probability 1/2, due to two possible illegal moves (up and left), each with probability 1/4, that bounce him back to cell 1.</li>
    <li> If he is in cell 10, he will teleport to cell 5 with probability 1/2. With probability 1/2, he will attempt to move up, down, left, or right. Moving down will bounce him back to cell 10. Thus, the transition probabilities out of cell 10 under this "mostly random walk" policy are
        <br><br>
        $$
        \begin{array}{lcl}
        p(10,5) &=& 1/2\\
        p(10,6) &=& (1-1/2)(1/4) = 1/8\\
        p(10,9) &=& (1-1/2)(1/4) = 1/8\\
        p(10,10) &=& (1-1/2)(1/4) = 1/8 \,\,\, \mbox{(bounce off of blocked cell)}\\
        p(10,11) &=& (1-1/2)(1/4) = 1/8
        \end{array}
        $$

<hr>
<b>Part A)</b> In the next several cells, we will begin the solution towards finding the optimal policy for Marvin to maximize his expected rewards over the time. <b>Fair warning-- there is going to be a problem for you to fix</b> so pay attention to what is in each cell you are asked to run!

<hr>
We begin by setting the discount factor and defining the rewards vector. For example, the reward associated with Marving being in state 2 is

$$
r(s_{2}) = \frac{1}{4}(-1) + \frac{1}{4}(-1)+\frac{1}{4}(0)+\frac{1}{4}(5) = \frac{3}{4}.
$$

These terms are associated with moves up, down, left, and right, respectively.

As a second example, the reward associated with Marvin being in state 3 is

$$
r(s_{3}) = (1)(0) = 0
$$

because, once in state 3, he will teleport to state 8 with probability 1 and there is no reward for making this move.

Does this seem right to you? Isn't state 3 a "high value state" because it contains the cheese? Rewards and the state values are different things, so let's proceed for now.

In the next cell, we set up the full rewards vector. You may want to try one or two more rewards computations of your own and check them against the corresponding elements in the given rewards vector.

In [2]:
# Set the discount factor
gamma = 0.95

# Define the rewards vector 
r = np.array([-1/2,3/4,0,3/4,-1/2,1,-1/4,-1/4,-1/4,-1/8,1/4,-1/2,-1/2,0])

We now set up the state transition probabilities under the "mostly random walk" policy. For example, when in state 1, Marvin will attempt to go up, down, left, or right with probability 1/4 each. Moves up or to the left, will bounce him back to cell 1. Thus, when in state 1, Marvin will stay in state 1 with probability 1/2 or move to either state 2 or 5 with probability 1/4 each.

Pick a couple more rows and check the matrix values given in the next cell before running the cell.

In [3]:
# Define the P matrix. Don't forget to use zero-based indexing! 
P = np.zeros((14,14))
P[0,[0,1,4]] = np.array([0.5,0.25,0.25])
P[1,[0,1,2]] = np.array([0.25,0.5,0.25])
P[2,7] = 1
P[3,[2,3,6]] = np.array([0.25,0.5,0.25])  
P[4,[0,4,7]] = np.array([0.25,0.5,0.25])
P[5,[2,5,6,9]] = 0.25
P[6,[3,5,6,10]] = 0.25
P[7,[4,7,8,11]] = 0.25
P[8,[7,8,9,12]] = 0.25
P[9,[4,5,8,9,10]] = np.array([0.5,0.125,0.125,0.125,0.125])
P[10,[6,9,10,13]] = 0.25
P[11,[7,11,12]] = np.array([0.25,0.5,0.25])
P[12,[8,11,12]] = np.array([0.25,0.25,0.5])
P[13,11] = 1

We are now ready to find the state value function $v_{\pi}(s)$ for all 14 states. We will use the equation

$$
\vec{v}_{\pi} = (I-\gamma {\bf{P}})^{-1} \vec{r}_{\pi}.
$$

In [4]:
# Define a 14x14 identity matrix
ident = np.identity(14)

# Compute the state value function
v = np.matmul(np.linalg.inv(ident-gamma*P),r)

In [5]:
# For easier visualization, will organize values into a matrix but
# first need to fill in those blocked cells with something
# We will use "np.nan" which stands for "not a number"  but we could choose 
# anything distinguishable like -99.9999.
# Recall that v[start:stop] will return v values with indices from "start"

v_visual = np.concatenate((v[0:5],[np.nan],v[5:13],[np.nan],v[13:14]))
v_visual.reshape((4,4))

array([[-5.59180767, -3.86458674, -6.10885775, -3.06387102],
       [-6.39098811,         nan, -3.50158532, -3.8218045 ],
       [-6.43037658, -6.20745252, -5.52179588, -4.65191601],
       [-6.993821  , -6.92438564,         nan, -6.64412995]])

Consider writing down the new greedy policy (the grid with arrows determined by the value vector) on paper, for your reference. Remember, you do not need arrows in cells 3 or 14 because the movement there is always deterministic teleportation. Do you see a problem? The cheese and lettuce states (cells 3 and 14) seem like they should be high value states, but they appear to have low values when compared to neighboring cells. Indeed, your policy "arrow" grid should, at this point, appear to want to send Marvin to cell 2 when he is in cell 1 and to send Marvin to cell 1 when he is in cell 2! He is not being pushed towards the lettuce and cheese at all from these states! What went wrong here?

Answer: The problem is with the rewards structure. As noted, the reward associated being in state 3 (the "cheese state") is 0 because it is a function of what Marvin can get when he moves out of the cheese state and not into it. To fix this, we will change the rewards structure so that 

<ul>
    <li>Marvin gets nothing for entering cell 3 but gets the reward of +5 when teleporting out.</li>
    <li> He will get nothing for entering cell 14 but gets the reward of +2 when teleporting out.
</ul>  

<b> Part B)</b> In the next cell, redefine the rewards vector to reflect the changes to the rewards given at the end of the previous cell. Call your rewards vector <code>r_b</code>.

In [19]:
# Define the rewards vector
r_b = np.array([0,0,5,0,0,0,0,0,0,0,0,0,0,2])
# your code here


In [20]:
# Hidden Test Cell (Run this cell!)
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

In the next cell, we define the transition probability matrix ${\bf{P}}$. We will call it P_b because it is the transition matrix being used in part (b) of this exercise. However, we are just restarting the problem with a better rewards structure and this does not represent a policy iteration.

In [21]:
# Define the P matrix 
P_b = P

In the next cell, compute the state value function v for all states. Call the vector of values v_b. 

In [26]:
# Define v
I = np.eye(14)
v_b = np.linalg.inv(I - gamma * P) @ r_b



In [27]:
# Hidden Test Cell (Run this cell!)
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

In [28]:
# Run this cell to better visualize v_b in terms of grid neighboring state values.
v_visual = np.concatenate((v_b[0:5],[np.nan],v_b[5:13],[np.nan],v_b[13:14]))
v_visual.reshape((4,4))

array([[-2.84510933,  0.28520761,  0.31767351,  1.41311562],
       [-4.4691335 ,         nan,  0.33122489, -0.35183899],
       [-4.92876472, -4.5809419 , -3.11295461, -1.82129726],
       [-5.72122186, -5.61288361,         nan, -3.43516077]])

Which of the follow grids with arrows represents the greedy policy at this point? The answer is either 1, 2, 3, or 4. Store your answer in the next cell in a variable called <code>policy_b</code>.

<br>
<img src="four_policies_b.png" width="700"> <br>

In [28]:
policy_b = 3



In [29]:
# Hidden Test Cell (Run this cell!)
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

<b>Part C)</b> Let's continue through another policy iteration. In the next cell, enter the next iteration of the rewards vector. Call this r_c (This is the second iteration of the rewards vector but we are calling it r.c because we are in part (c) of this problem.)

In [30]:
# Define the rewards vector
r_c = np.array([0,0,5,0,0,0,0,0,0,0,0,0,0,2])

# your code here

In [31]:
# Hidden Test Cell (Run this cell!)
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

In the next cell, define the transition probability matrix ${\bf{P}}$. Call this matrix P_c. You can copy and paste the code to define the original ${\bf{P}}$ to get you started, but you will have to make changes. Don't forget that, if Marvin is in cell 10, he will always have a possibility of teleporting to cell 5 with probability 1/2. With probability 1/2, he will follow the policy that you picked out for policy_b.

In [32]:
# Define the P matrix. Don't forget to use zero-based indexing! 

P_c = np.zeros((14,14))

# Deterministic moves for non-special states
# state 1 -> 2
P_c[0,1] = 1.0

# state 2 -> 3
P_c[1,2] = 1.0

# state 3: teleport to 8
P_c[2,7] = 1.0

# state 4 -> 3
P_c[3,2] = 1.0

# state 5 -> 1
P_c[4,0] = 1.0

# state 6 -> 3
P_c[5,2] = 1.0

# state 7 -> 4
P_c[6,3] = 1.0

# state 8 -> 9
P_c[7,8] = 1.0

# state 9 -> 10
P_c[8,9] = 1.0

# state 10: 1/2 teleport to 5, 1/2 move up to 6
P_c[9,4] = 0.5   # to state 5
P_c[9,5] = 0.5   # to state 6

# state 11 -> 7
P_c[10,6] = 1.0

# state 12 -> 8
P_c[11,7] = 1.0

# state 13 -> 9
P_c[12,8] = 1.0

# state 14: teleport to 12
P_c[13,11] = 1.0

In [33]:
# Hidden Test Cell
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

In the next cell, compute the state value function v for all states. Call the vector of values v_c.

In [34]:
# Define v
v_c = v_c = np.linalg.inv(ident - gamma * P_c) @ r_c



In [35]:
# Hidden Test Cell (Run this cell!)
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

In [36]:
# Run this cell to better visualize v_b in terms of grid neighboring state values.
v_visual = np.concatenate((v_c[0:5],[np.nan],v_c[5:13],[np.nan],v_c[13:14]))
v_visual.reshape((4,4))

array([[17.09663143, 17.99645414, 18.94363594, 17.99645414],
       [16.24179986,         nan, 17.99645414, 17.09663143],
       [14.67751151, 15.45001212, 16.26317065, 16.24179986],
       [13.94363594, 14.67751151,         nan, 15.24645414]])

Which of the follow grids with arrows represents the greedy policy at this point? The answer is either 1, 2, 3, or 4. Store your answer in the next cell in a variable called <code>policy_c</code>.

<br>
<img src="four_policies_round2.png" width="700"> <br>

In [37]:
policy_c = 2

# your code here


In [38]:
# Hidden Test Cell (Run this cell!)
# NOTE: This cell contains hidden tests. You will not see whether you passed these tests until you submit your assignment.
# Any cell labeled "Hidden Test Cell" MAY have hidden tests.

This is the end of this lab. Note that we have not necessarily found Marvin's optimal policy yet. We need to continuing updating the rewards vector, the transition probability matrix, and ultimately the state value function until the values of the state value function have stabilized. In order to do this, it would be beneficial to express the states as state $(1,1)$ through state $(4,4)$ as opposed to $1$ through $14$, to store the state value and rewards vectors as matrices, and to automate the policy finding step. This last part is non-trivial from a coding persepctive since each cell could contain different numbers of arrows!